In [2]:
from xgboost import XGBRegressor 

In [3]:
%run preprocess_scikit.ipynb

model
VIANO 2.2 CDI                                                  4405
VIANO 2.0 CDI                                                  2931
COMBI 116 CDI                                                  2847
COMBI 113 CDI                                                  1968
VIANO 3.0 CDI                                                  1180
                                                               ... 
2008 1.6 e-Hdi FAP BVM6                                           1
A6 2.0 TDI (136ch) MULTITRONIC 8                                  1
PASSAT SW 1.6 TDI (105ch) CR FAP BMT BVM6                         1
MULTIVAN CONFORTLINE ET HIGHLINE 2.0 TDI (180ch) BlueMotion       1
ASX CLEARTEC 1.8 DI-D Invite / Intense / Instyle 4WD              1
Name: count, Length: 3141, dtype: int64
3141
car_class
MINIBUS                34570
BERLINE                 3307
BREAK                    933
TS TERRAINS/CHEMINS      819
COUPE                    661
CABRIOLET                389
MONOSPACE COMPACT

## Construction des modèles

Faisons d'abord les fonctions de prédictions et de Cross Validation

### Fonction pour le prétraitement des données de test

In [4]:
def preprocess(df_test):
    is_empty = (df_test['hc'].isnull() & df_test['hcnox'].notnull() & df_test['nox'].notnull())

    # Remplir les valeurs manquantes dans "hcnox" avec la somme de "hc" et "nox"
    df_test.loc[is_empty,'hc'] = df_test.loc[is_empty,'hcnox'] - df_test.loc[is_empty,'nox']
    df_test = df_test.drop(columns=["id","hcnox","model"])

    df_test[['gearbox', 'ratio']] = df_test["grbx_type_ratios"].str.split(' ', n=1, expand=True)
    df_test['ratio'] = df_test['ratio'].replace({'.': '0'})                                          #Remplacer les points par zéro
    df_test['ratio'] = pd.to_numeric(df_test['ratio'], errors='coerce').fillna(0)
    df_test = df_test.drop(columns="grbx_type_ratios")

    is_not_abundant = ~df_test["brand"].isin(["MERCEDES", "VOLKSWAGEN"])
    df_test.loc[is_not_abundant, "brand"] = "AUTRE"

    return df_test

### Fonction pour avoir les prédictions

In [5]:
def get_prediction(X_val, y_val, model):
    pred = model.predict(X_val) # Your code here
    
    # Calculate MAE
    mae = mean_absolute_error(y_val, pred)

    # Uncomment to print MAE
    print("Mean Absolute Error:" , mae)

### Fonction pour la Cross Validation

In [6]:
def validation(model,nb_validate):    
    cv_results = cross_validate(
        model, data, target, cv=nb_validate, return_estimator=True,           #nb_validate validations croisées
        return_train_score=True, scoring="neg_mean_squared_error"
    )
    scores_train=-cv_results["train_score"]
    scores_test=-cv_results["test_score"]
    print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données d'entrainement:\n"
        f"{scores_train.mean():.3f} ± {scores_train.std():.3f}")

    print(f"Erreur quadratique moyenne du modèle de régression linéaire sur les données de test:\n"
        f"{scores_test.mean():.3f} ± {scores_test.std():.3f}")    

## -- XGBoost Pipeline

In [9]:
xg_regressor = XGBRegressor(n_estimators=10000, 
                     random_state=0, 
                     learning_rate=0.01,
                     max_depth=100,
                     enable_categorical=True)

xg_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessors),
        ('regressor', xg_regressor)
    ])

### Hyper parameter Tuning du XGBoost

In [22]:
param_grid = {
    'regressor__n_estimators': [1500, 2000, 2300, 2500, 3000],
    'regressor__max_depth': [3, 6, 9, 12],
    'regressor__learning_rate': [0.07, 0.1, 0.13, 0.15, 0.2]
}

RandomizedSearchCV a été préféré à GridSearchCV car GridSearchCV teste toutes les combinaisons ce qui prend énormément de temps.

In [24]:
random_cv = RandomizedSearchCV(
    estimator=xg_pipeline,
    param_distributions=param_grid,
    cv=3,                                           # Trois validations croisées pour déterminer les meilleures hyper paramètres
    n_iter=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=5,
    return_train_score=True,
    random_state=42
)

In [21]:
xg_pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessor', 'regressor', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__verbose_feature_names_out', 'preprocessor__categorical preprocessing', 'preprocessor__numerical preprocessing', 'preprocessor__categorical preprocessing__memory', 'preprocessor__categorical preprocessing__steps', 'preprocessor__categorical preprocessing__verbose', 'preprocessor__categorical preprocessing__imputer', 'preprocessor__categorical preprocessing__encoder', 'preprocessor__categorical preprocessing__imputer__add_indicator', 'preprocessor__categorical preprocessing__imputer__copy', 'preprocessor__categorical preprocessing__imputer__fill_value', 'preprocessor__categorical preprocessing__imputer__keep_empty_features', 'preprocessor__categorical preprocessing__imputer__missing_values', 'preprocessor__categorical preprocessin

In [25]:
random_cv.fit(x_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('categorical '
                                                                               'preprocessing',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(fill_value='missing',
                                                                                                              strategy='constant')),
                                                                                               ('encoder',
                                                                                                OneHotEncoder(handle_unknown='ignore'))]),
                                                                               ['brand',
                                                                                'car_class',
                                                                                'range',
                                                                                'fuel_type',
                                                                                'hybrid',
                                                                                'gearbox']),
                                                                              ('numerical '
                                                                               'preprocess...
                                                           multi_strategy=None,
                                                           n_estimators=10000,
                                                           n_jobs=None,
                                                           num_parallel_tree=None,
                                                           random_state=0, ...))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'regressor__learning_rate': [0.07, 0.1,
                                                                     0.13, 0.15,
                                                                     0.2],
                                        'regressor__max_depth': [3, 6, 9, 12],
                                        'regressor__n_estimators': [1500, 2000,
                                                                    2300, 2500,
                                                                    3000]},
                   random_state=42, return_train_score=True,
                   scoring='neg_root_mean_squared_error', verbose=5)

In [26]:
random_cv.best_params_

{'regressor__n_estimators': 3000,
 'regressor__max_depth': 3,
 'regressor__learning_rate': 0.13}

### Testons notre modèle et création du fichier pour le submit sur Kaggle

In [27]:
xg_pipeline.fit(data,target)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical preprocessing',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['brand', 'car_class',
                                                   'range', 'fuel_type',
                                                   'hybrid', 'gearbox']),
                                                 ('numerical preprocessing',
                                                  Pipeline(steps=[('imputer',
                                                                   Simple...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.01,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=100, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=10000, n_jobs=None,
                              num_parallel_tree=None, random_state=0, ...))])

In [28]:
df_test_processed = preprocess(df_test)
predictions  = xg_pipeline.predict(df_test_processed)

In [29]:
submit = pd.DataFrame({'id': df_test["id"], 'co2': predictions})
submit.to_csv("predictions.csv", index=False)